# Preparation

In [ ]:
!pip install pygame

     |████████████████████████████████| 21.8 MB 12.1 MB/s 


In [ ]:
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

pygame 2.1.2 (SDL 2.0.16, Python 3.7.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


<Surface(640x480x32 SW)>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Thesis_Simulation2/Vehicle rate exploration/No connection

Mounted at /content/gdrive
/content/gdrive/MyDrive/Thesis_Simulation2/Vehicle rate exploration/No connection


In [ ]:
import math
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(0,'../..')

from trafficSimulator import *
from SimulationAnalysis import utility_summary, compute_utility, update_utility_df

## Simulation setup

### Simulation params setup

In [ ]:
all_routes = [[0, 1], [2, 3]]

v_max = 9
s0 = 2
T = 1
b_max = 2
a_max = 1

vehicle_specs = {'v_max': v_max, # Desired speed
                 's0': s0, # Safe bumper-to-bumper distance
                 'T': T, # Time gap
                 'b_max': b_max, # Deceleartion
                 'a_max': a_max, # Acceleration
                 'l' : 3
                }

fast_track_factor, slow_track_factor = 1, 0.33
stop_distance = 25
fast_wait_time = 4
slow_wait_time = 4
vehicle_limit = 80

df = pd.DataFrame(columns=['Vehicle_label', 'Road_order', 
                           'Total_time', 'Leading_vehicles',
                           'Stopped_time', 'Stop_while_front'])

vehicle_preferences = dict(zip(range(vehicle_limit), 
                               [[0.5, 0.5]]*vehicle_limit))

### Topology setup

In [ ]:
def parallel_line(current_coordinates, distance):
    """
    Returns the coordinate sets of lines that are parallel to the given line
    with the speicified distance
    
    Input:
        current_coordinates: two set of x and y coordinates that mark the start
                             and end of the given line
        distance: the distance between the given line and the parallel lines to
                  to be output
    """
    
    x1, y1 = current_coordinates[0]
    x2, y2 = current_coordinates[1]
    k = (y1-y2)/(x1-x2)
    b = y1 - k*x1
    
    cross_x = -b/k
    hypo = math.sqrt(b**2+cross_x**2)
    
    if k>0:
        new_coord_1 = ((x2+distance*abs(b)/hypo, y2-distance*abs(cross_x)/hypo),
                       (x1+distance*abs(b)/hypo, y1-distance*abs(cross_x)/hypo))

        new_coord_2 = ((x2-distance*abs(b)/hypo, y2+distance*abs(cross_x)/hypo),
                       (x1-distance*abs(b)/hypo, y1+distance*abs(cross_x)/hypo))
        
    else:
        new_coord_1 = ((x2+distance*abs(b)/hypo, y2+distance*abs(cross_x)/hypo),
                       (x1+distance*abs(b)/hypo, y1+distance*abs(cross_x)/hypo))
    
        new_coord_2 = ((x2-distance*abs(b)/hypo, y2-distance*abs(cross_x)/hypo),
                       (x1-distance*abs(b)/hypo, y1-distance*abs(cross_x)/hypo))
    
    return new_coord_1, new_coord_2

In [ ]:
left, right = -75*math.sqrt(3), 75*math.sqrt(3)
bottom, top = -75, 75

left_bottom_outbound = ((left+2, 4), (-5, top-2),
                        slow_track_factor, stop_distance, slow_wait_time)
bottom_right_outbound = ((5, top-2), (right-2, 4),
                         fast_track_factor, stop_distance, fast_wait_time)

left_top_outbound = ((left+2, -4), (-5, bottom+2),
                     fast_track_factor, stop_distance, fast_wait_time)
top_right_outbound = ((5, bottom+2), (right-2, -4),
                      slow_track_factor, stop_distance, slow_wait_time)

bottom_left_inbound = parallel_line(current_coordinates=left_bottom_outbound, 
                                    distance=2.5)[0]
right_bottom_inbound = parallel_line(current_coordinates=bottom_right_outbound, 
                                     distance=2.5)[1]

top_left_inbound = parallel_line(current_coordinates=left_top_outbound, 
                                 distance=2.5)[0]
right_top_inbound = parallel_line(current_coordinates=top_right_outbound, 
                                  distance=2.5)[1]

### Utility params setup

In [ ]:
# Smaller delta encourages more exploration
delta = 0.25

# Penalty proportion for delay caused
alpha = 0

# # Interpolation factor between money and time
# gamma_mean, gamma_var = 5, 0.5

# # Level of risk aversion
# eta_mean, eta_var = 1, 0.2

## Function needed for running simulation

In [ ]:
 def run_simulation(round_number, all_routes, vehicle_limit, df, vehicle_preferences,
                    vehicle_rate, vehicle_specs):
    """
    Executes the simulation and returns certain data of the simulation.
    
    Input:
        round_number (integer): the number of the current simulation
        all_routes (list): all possible paths that a vehicle can take
        vehicle_limit (integer): the total number of vehicles in the simulation
        df (pandas DataFrame): a dataframe for storing data from the simulation
        vehicle_preferences (dict): store vehicle label as keys and their probabilities
                                    of choosing each of the possible routes as values
        vehicle_rate (integer): frequency of generating new vehicles
        vehicle_specs (dict): parameters related to the vehicles
    """
    global left_bottom_outbound
    global bottom_right_outbound
    global left_top_outbound
    global top_right_outbound
    global bottom_left_inbound
    global right_bottom_inbound
    global top_left_inbound
    global right_top_inbound
    
    records = df.copy()
    sim = Simulation({
        'round_number': round_number,
        'all_routes': all_routes, # All possible (and reasonable) routes
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'records': records, # Table that will capture the needed vehicle-related info
        'vehicle_preferences': vehicle_preferences
        # A dictionary that stores the vehicles' labels as keys and their list of probabilities
        # of choosing each possible route as values
        # For the initial few rounds all vehicles choose all routes with equal weights.
        # For more complicated topologies some other algorithms will be needed as all possible
        # routes are not as explicit as in here.
        })

    sim.create_roads([
        ## Key routes
        left_bottom_outbound, # Road #0
        bottom_right_outbound, # Road #1

        left_top_outbound, # Road #2
        top_right_outbound, # Road #3

        ## Curved corners
        # Note: in the simulation, the vehicles will not actually go pass these
        # routes; they are more for aesthetic purposes

        # Outbound corners
        *curve_road(left_bottom_outbound[1], 
                    (bottom_right_outbound[0][0], bottom_right_outbound[0][1]+0.01), 
                    (0, top), 16), # Outbound bottom corner

        *curve_road(left_top_outbound[1], 
                    (top_right_outbound[0][0], top_right_outbound[0][1]+0.01), 
                    (0, bottom), 16), # Outbound top corner

        *curve_road(left_bottom_outbound[0], 
                    (left_top_outbound[0][0]+0.01, left_top_outbound[0][1]), 
                    (left, 0), 16), # Outbound left corner

        *curve_road(bottom_right_outbound[1], 
                    (top_right_outbound[1][0]+0.01, top_right_outbound[1][1]), 
                    (right, 0), 16), # Outbound right corner
    ])


    sim.create_gen({
        'vehicle_rate': vehicle_rate, # Rate of generating new vehicles
        'vehicle_limit': vehicle_limit, # Total number of vehicles in simulation
        'vehicles': vehicle_specs
        })

    # Start simulation
    win = Window(sim)
    win.zoom = 4
    new_records = win.run(steps_per_update=5)
    
    return new_records

# Start running

In [ ]:
def simulation_with_changing_rate(vehicle_rates, rounds,
                                  all_routes, 
                                  vehicle_limit, df, 
                                  vehicle_preferences,
                                  vehicle_specs):

    for r in range(len(vehicle_rates)):
        rate = vehicle_rates[r]
        
        # Run the 1st round
        record_df = run_simulation(1, all_routes, vehicle_limit, df, vehicle_preferences,
                                     rate, vehicle_specs)

        # Simulation Analysis of the 1st round
        record_df['Round_number'] = 1
        record_df['vehicle_rate'] = rate

        
        utility_df = pd.read_csv(f"../Zero alpha/Saved_data/Utility_data/20/Round_1.csv")

        utility_df['Vehicle_label'] = list(range(len(record_df.Vehicle_label.unique())))
        utility_df['Routes_taken'] = [[]] * len(utility_df)
        utility_df['Utilities'] = [[0] * len(all_routes)] * len(utility_df)
        utility_df['Probabilities'] = [[0] * len(all_routes)] * len(utility_df)  

        utility_df = utility_df.sort_values('Vehicle_label', ascending=True).reset_index(drop=True)            

        utility_tmp = utility_summary(all_routes, record_df)
        utility_df = update_utility_df(all_routes,
                                       delta, 
                                       alpha, 
                                       record_df=record_df,
                                       df=utility_df, 
                                       utility_tmp=utility_tmp)

        # Export
#         record_df.to_csv("Saved_data/Simulation_records/Round_1.csv", index=False)

        if not os.path.exists(f'Saved_data/Utility_data/{rate}'):
            os.makedirs(f'Saved_data/Utility_data/{rate}')
        utility_df.to_csv(f"Saved_data/Utility_data/{rate}/Round_1.csv", index=False)


        # Updated probabilities for the 2nd round:
        vehicle_preferences = dict(zip(range(vehicle_limit), utility_df.Probabilities.to_list()))

        for s in range(2, rounds+1):
            record_tmp = run_simulation(s, all_routes, 
                                        vehicle_limit, 
                                        df, vehicle_preferences,
                                        rate, vehicle_specs)
            record_tmp['Round_number'] = s
            record_tmp['vehicle_rate'] = rate

            record_df = pd.concat([record_df, record_tmp], ignore_index=True)

            utility_tmp = utility_summary(all_routes, record_df)
            utility_df = update_utility_df(all_routes,
                                           delta, 
                                           alpha, 
                                           record_df=record_df,
                                           df=utility_df, 
                                           utility_tmp=utility_tmp,
                                           round_number=s)

            vehicle_preferences = dict(zip(range(vehicle_limit), 
                                           utility_df.Probabilities.to_list()))
            utility_df.to_csv(f"Saved_data/Utility_data/{rate}/Round_{s}.csv", index=False)
                                           

        # Export
        record_df.to_csv(f"Saved_data/Simulation_records/VehicleRate_{rate}_Round_{s}.csv", index=False)
        

#         utility_tmp.to_csv(f"Saved_data/Simulation_records/Summary.csv", index=False)

In [ ]:
%%time
simulation_with_changing_rate(vehicle_rates=np.arange(44,50,2), 
                              rounds=50,
                              all_routes=all_routes, 
                              vehicle_limit=vehicle_limit, df=df, 
                              vehicle_preferences=vehicle_preferences,
                              vehicle_specs=vehicle_specs)

CPU times: user 6h 39min 28s, sys: 10min 40s, total: 6h 50min 9s
Wall time: 6h 37min 39s
